In [2]:


import os
import random

random_seed = 32
random.seed (random_seed)

import pandas as pd

housing = pd.read_csv("housing.csv")

housing.head


<bound method NDFrame.head of        longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0        -122.23     37.88                41.0        880.0           129.0   
1        -122.22     37.86                21.0       7099.0          1106.0   
2        -122.24     37.85                52.0       1467.0           190.0   
3        -122.25     37.85                52.0       1274.0           235.0   
4        -122.25     37.85                52.0       1627.0           280.0   
...          ...       ...                 ...          ...             ...   
20635    -121.09     39.48                25.0       1665.0           374.0   
20636    -121.21     39.49                18.0        697.0           150.0   
20637    -121.22     39.43                17.0       2254.0           485.0   
20638    -121.32     39.43                18.0       1860.0           409.0   
20639    -121.24     39.37                16.0       2785.0           616.0   

       population  ho

In [3]:
housing.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [4]:
housing.isna().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

In [88]:
#Data is incomplete we could use the imputer, avg mean, mediana asignar un valor

In [5]:
housing["ocean_proximity"].unique()

array(['NEAR BAY', '<1H OCEAN', 'INLAND', 'NEAR OCEAN', 'ISLAND'],
      dtype=object)

In [6]:

target_variable = "median_house_value"
explanatory_variables = [var for var in housing.columns.values if var != "median_house_value"]
print (target_variable)
print(explanatory_variables)
from sklearn.model_selection import train_test_split 

median_house_value
['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'ocean_proximity']


In [92]:
## Defines training and test

In [7]:
X = housing[explanatory_variables]
y = housing[target_variable]

X_train, X_test, y_train, y_test = train_test_split (X, y, test_size=.2, random_state=random_seed)

In [8]:

print (X_train.shape)
print (y_train.shape)
print (y_test.shape)
print (X_test.shape)

(16512, 9)
(16512,)
(4128,)
(4128, 9)


In [9]:


#
# Pre processing pipeline 
#
#
categorical_variables = ["ocean_proximity"]
numerical_variables = [var for var in explanatory_variables if var not in categorical_variables]
print (categorical_variables)
print (numerical_variables)

['ocean_proximity']
['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income']


In [10]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

numerical_pipeline = Pipeline ([("imputer", SimpleImputer(strategy="constant", fill_value=0)), ("scaler", StandardScaler())])
                                                

In [11]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

preprocessing_pipeline = ColumnTransformer ([("numerical_pipeline", numerical_pipeline, numerical_variables),
                                             ("encoder", OneHotEncoder(), categorical_variables)
                                            ])

In [98]:
X_train_transformed = preprocessing_pipeline.fit_transform (X_train)
X_train_transformed

array([[-0.61053778,  1.34571519, -0.44773646, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.82224293, -0.93015644,  0.50551543, ...,  0.        ,
         0.        ,  0.        ],
       [-1.34440107,  1.01791475, -1.48042601, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [-1.78871286,  1.63605272,  0.34664011, ...,  0.        ,
         0.        ,  0.        ],
       [-0.91007382,  0.34826528,  0.82326606, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.842212  , -0.87864494,  0.42607777, ...,  0.        ,
         0.        ,  0.        ]])

In [12]:
import numpy as np
#np.sum(np_isnan(X_train_transformed))


In [13]:
from sklearn.linear_model import LinearRegression
regressor_pipeline = Pipeline ([("preprocessing", preprocessing_pipeline), ("regressor", LinearRegression())])

In [14]:
regressor_pipeline.fit (X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('numerical_pipeline',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value=0,
                                                                                 strategy='constant')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['longitude', 'latitude',
                                                   'housing_median_age',
                                                   'total_rooms',
                                                   'total_bedrooms',
                                                   'population', 'households',
                                                   'median_income']),
                                                 ('encoder', OneHotEncoder(),
                                                  ['ocean_proximity'])])),
                ('regressor', LinearRegression())])

In [15]:
#Evaluate
regressor_pipeline.score(X_train, y_train)

0.6481736095567955

In [16]:
regressor_pipeline.score(X_test, y_test)

0.630409442879158

In [17]:
from sklearn.metrics import mean_squared_error as mse
np.sqrt(mse(y_train, regressor_pipeline.predict(X_train)))

68215.06943016544